In [1]:
import gensim
import pandas as pd
import numpy as np
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim_models
from collections import Counter
from string import punctuation
from razdel import tokenize as razdel_tokenize
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.decomposition import TruncatedSVD, NMF, PCA, LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler 
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
from stop_words import get_stop_words

morph = MorphAnalyzer()
warnings.filterwarnings("ignore")
ru_stopwords = get_stop_words('ru')

C:\Users\huawei\anaconda3\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
C:\Users\huawei\anaconda3\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
C:\Users\huawei\anaconda3\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
C:\Users\huawei\anaconda3\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw,

# Домашнее задание  № 5. Матричные разложения/Тематическое моделирование

### Задание № 1 (4 балла)

Попробуйте матричные разложения с 5 классификаторами - SGDClassifier, KNeighborsClassifier, MultinomialNB, RandomForest, ExtraTreesClassifier (про него подробнее почитайте в документации, он похож на RF). Используйте и NMF и SVD. Сравните результаты на кросс-валидации и выберите лучшее сочетание.

В итоге у вас должно получиться, как минимум 10 моделей (два разложения на каждый классификатор). Используйте 1 и те же параметры кросс-валидации. Параметры векторизации, параметры K в матричных разложениях, параметры классификаторов могут быть разными между экспериментами.

Можете взять поменьше данных, если все будет обучаться слишком долго (не ставьте параметр K слишком большим в NMF, иначе точно будет слишком долго)

In [2]:
data = pd.read_csv('toxic_comments.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14412 entries, 0 to 14411
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comment  14412 non-null  object 
 1   toxic    14412 non-null  float64
dtypes: float64(1), object(1)
memory usage: 225.3+ KB


In [6]:
data['toxic'].value_counts()

0.0    9586
1.0    4826
Name: toxic, dtype: int64

In [2]:
punctuation += '«»—-'

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 and word not in ru_stopwords and word not in punctuation]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)

In [7]:
data['comment_norm'] = data['comment'].apply(normalize)

In [396]:
def eval_table(X, y, pipeline, N=3):
    # зафиксируем порядок классов
    labels = list(set(y))
    
    # метрики отдельных фолдов будет хранить в табличке
    fold_metrics = pd.DataFrame(index=labels)
    # дополнительно также соберем таблицу ошибок
    errors = np.zeros((len(labels), len(labels)))
    
    # создаем стратегию кросс-валидации
    # shuffle=True (перемешивание) - часто критично важно указать
    # т.к. данные могут быть упорядочены и модель на этом обучится
    kfold = StratifiedKFold(n_splits=N, shuffle=True, random_state=42)
    
    for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
        # fit-predict как и раньше, но сразу пайплайном
        pipeline.fit(X[train_index], y[train_index])
        preds = pipeline.predict(X[test_index])
        
        # записываем метрику и индекс фолда
        fold_metrics[f'precision_{i}'] = precision_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'recall_{i}'] = recall_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)
        errors += confusion_matrix(y[test_index], preds, labels=labels, normalize='true')
    
    # таблица для усредненных значений
    # тут мы берем колонки со значениями и усредняем их
    # часто также все метрики сразу суммируют и в конце просто делят на количество фолдов
    # но мы тут помимо среднего также хотим посмотреть на стандартное отклонение
    # чтобы понять как сильно варьируются оценки моделей
    result = pd.DataFrame(index=labels)
    result['precision'] = fold_metrics[[f'precision_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['precision_std'] = fold_metrics[[f'precision_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['recall'] = fold_metrics[[f'recall_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['recall_std'] = fold_metrics[[f'recall_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['f1_std'] = fold_metrics[[f'f1_{i}' for i in range(N)]].std(axis=1).round(2)
    
    # добавим одну колонку со средним по всем классам
    result.loc['mean'] = result.mean().round(2)
    # проценты ошибок просто усредняем
    errors /= N
    
    return result, errors

Все параметры моделей сначала подбирал на обычном train_test_split, чтобы понять какие должны быть, чтобы модели показывали хорошее качество. А потом уже делал кроссвалидацию. На NMF большие значения взять не получилось, слишком много времени занимало это разложение, да и качество хуже было при увеличении этого параметра, поэтому в этом особо смысла и не было.

<b>SGDClassifier</b>

In [397]:
pipeline_bow = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), max_df=0.5, stop_words=ru_stopwords)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='log', alpha=0.00001, random_state=42))
])

pipeline_svd = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), max_df=0.5, stop_words=ru_stopwords)),
    ('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(100)),
    ('clf', SGDClassifier(loss='log', alpha=0.00001, random_state=42))
])

pipeline_nmf = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), max_df=0.5, stop_words=ru_stopwords)),
    ('tfidf', TfidfTransformer()),
    ('svd', NMF(20)),
    ('clf', SGDClassifier(loss='log', alpha=0.00001, random_state=42))
])

In [398]:
metrics_bow, errors_bow = eval_table(data['comment_norm'], data['toxic'], pipeline_bow)
metrics_svd, errors_svd = eval_table(data['comment_norm'], data['toxic'], pipeline_svd)
metrics_nmf, errors_nmf = eval_table(data['comment_norm'], data['toxic'], pipeline_nmf)

In [399]:
metrics_bow

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.86,0.01,0.96,0.00,0.91,0.00
1.0,0.89,0.01,0.69,0.02,0.77,0.01
mean,0.88,0.01,0.82,0.01,0.84,0.00


In [400]:
metrics_svd

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.84,0.04,0.91,0.07,0.87,0.01
1.0,0.80,0.08,0.64,0.12,0.70,0.04
mean,0.82,0.06,0.78,0.10,0.78,0.02


In [401]:
metrics_nmf

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.76,0.08,0.92,0.11,0.82,0.01
1.0,0.77,0.12,0.38,0.30,0.45,0.23
mean,0.76,0.10,0.65,0.20,0.64,0.12


In [34]:
errors_bow

array([[0.98894131, 0.01105869],
       [0.78969413, 0.21030587]])

In [35]:
errors_svd

array([[0.94325039, 0.05674961],
       [0.6663919 , 0.3336081 ]])

In [402]:
errors_nmf

array([[0.92000029, 0.07999971],
       [0.62135652, 0.37864348]])

<b>Лучший результат показал BOW</b>

<hr>

<b>KNeighborsClassifier</b>

In [403]:
pipeline_bow = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), max_df=0.75, stop_words=ru_stopwords)),
    ('tfidf', TfidfTransformer()),
    ('clf', KNeighborsClassifier(n_neighbors=20, weights='distance', metric='euclidean'))
])

pipeline_svd = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), max_df=0.75, stop_words=ru_stopwords)),
    ('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(10)),
    ('clf', KNeighborsClassifier(n_neighbors=20, weights='distance', metric='euclidean'))
])

pipeline_nmf = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), max_df=0.5, stop_words=ru_stopwords)),
    ('tfidf', TfidfTransformer()),
    ('svd', NMF(10)),
    ('clf', KNeighborsClassifier(n_neighbors=20, weights='distance', metric='euclidean'))
])

In [404]:
metrics_bow, errors_bow = eval_table(data['comment_norm'], data['toxic'], pipeline_bow)
metrics_svd, errors_svd = eval_table(data['comment_norm'], data['toxic'], pipeline_svd)
metrics_nmf, errors_nmf = eval_table(data['comment_norm'], data['toxic'], pipeline_nmf)

In [405]:
metrics_bow

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.80,0.03,0.97,0.01,0.88,0.01
1.0,0.90,0.02,0.53,0.08,0.66,0.06
mean,0.85,0.02,0.75,0.04,0.77,0.03


In [406]:
metrics_svd

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.80,0.00,0.93,0.01,0.86,0.0
1.0,0.79,0.02,0.53,0.00,0.63,0.0
mean,0.80,0.01,0.73,0.00,0.74,0.0


In [407]:
metrics_nmf

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.79,0.02,0.91,0.01,0.85,0.01
1.0,0.74,0.04,0.52,0.04,0.61,0.04
mean,0.76,0.03,0.72,0.02,0.73,0.02


In [408]:
errors_bow

array([[0.97026859, 0.02973141],
       [0.47472924, 0.52527076]])

In [409]:
errors_svd

array([[0.92729014, 0.07270986],
       [0.46891771, 0.53108229]])

In [410]:
errors_nmf

array([[0.90913668, 0.09086332],
       [0.47699533, 0.52300467]])

<b>Лучший результат показал BOW</b>

<hr>

<b>MultinomialNB</b>

In [435]:
pipeline_bow = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,1), max_df=0.75, stop_words=ru_stopwords)),
    ('clf', MultinomialNB(alpha=0.3))
])

pipeline_svd = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,1), max_df=0.75, stop_words=ru_stopwords)),
    ('svd', TruncatedSVD(50)),
    ('scaler', MinMaxScaler()),
    ('clf', MultinomialNB(alpha=0.3))
])

pipeline_nmf = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,1), max_df=0.75, stop_words=ru_stopwords)),
    ('svd', NMF(5)),
    ('scaler', MinMaxScaler()),
    ('clf', MultinomialNB(alpha=0.3))
])

In [412]:
metrics_bow, errors_bow = eval_table(data['comment_norm'], data['toxic'], pipeline_bow)
metrics_svd, errors_svd = eval_table(data['comment_norm'], data['toxic'], pipeline_svd)
metrics_nmf, errors_nmf = eval_table(data['comment_norm'], data['toxic'], pipeline_nmf)

In [413]:
metrics_bow

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.89,0.0,0.93,0.00,0.91,0.00
1.0,0.85,0.0,0.77,0.01,0.81,0.01
mean,0.87,0.0,0.85,0.00,0.86,0.00


In [414]:
metrics_svd

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.67,0.0,1.0,0.0,0.8,0.0
1.0,0.00,0.0,0.0,0.0,0.0,0.0
mean,0.34,0.0,0.5,0.0,0.4,0.0


In [415]:
metrics_nmf

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.67,0.0,1.0,0.0,0.8,0.0
1.0,0.00,0.0,0.0,0.0,0.0,0.0
mean,0.34,0.0,0.5,0.0,0.4,0.0


In [416]:
errors_bow

array([[0.92958471, 0.07041529],
       [0.23124575, 0.76875425]])

In [417]:
errors_svd

array([[1., 0.],
       [1., 0.]])

In [418]:
errors_nmf

array([[1., 0.],
       [1., 0.]])

<b>Лучший результат показал BOW, матричные разложения не применимы для MultinomialNB</b>

<hr>

<b>RandomForest</b>

In [419]:
pipeline_bow = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,1), stop_words=ru_stopwords)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(max_depth=120, min_samples_leaf=4, n_estimators=400, random_state=42))
])

pipeline_svd = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,1), stop_words=ru_stopwords)),
    ('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(50)),
    ('clf', RandomForestClassifier(max_depth=120, min_samples_leaf=4, n_estimators=400, random_state=42))
])

pipeline_nmf = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,1), stop_words=ru_stopwords)),
    ('tfidf', TfidfTransformer()),
    ('svd', NMF(10)),
    ('clf', RandomForestClassifier(max_depth=120, min_samples_leaf=4, n_estimators=400, random_state=42))
])

In [420]:
metrics_bow, errors_bow = eval_table(data['comment_norm'], data['toxic'], pipeline_bow)
metrics_svd, errors_svd = eval_table(data['comment_norm'], data['toxic'], pipeline_svd)
metrics_nmf, errors_nmf = eval_table(data['comment_norm'], data['toxic'], pipeline_nmf)

In [421]:
metrics_bow

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.78,0.00,0.98,0.00,0.87,0.00
1.0,0.93,0.01,0.44,0.01,0.59,0.01
mean,0.86,0.00,0.71,0.00,0.73,0.00


In [422]:
metrics_svd

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.83,0.01,0.93,0.01,0.87,0.01
1.0,0.81,0.02,0.61,0.02,0.70,0.02
mean,0.82,0.02,0.77,0.02,0.78,0.02


In [423]:
metrics_nmf

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.80,0.00,0.90,0.00,0.85,0.00
1.0,0.74,0.01,0.57,0.01,0.64,0.01
mean,0.77,0.00,0.74,0.00,0.74,0.00


In [424]:
errors_bow

array([[0.98351751, 0.01648249],
       [0.56361359, 0.43638641]])

In [425]:
errors_svd

array([[0.92676739, 0.07323261],
       [0.38603311, 0.61396689]])

In [426]:
errors_nmf

array([[0.8972463 , 0.1027537 ],
       [0.43307108, 0.56692892]])

<b>Лучший результат показал BOW + SVD</b>

<hr>

<b>ExtraTreesClassifier</b>

In [427]:
pipeline_bow = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,1), stop_words=ru_stopwords)),
    ('tfidf', TfidfTransformer()),
    ('clf', ExtraTreesClassifier(max_depth=60, min_samples_leaf=3, n_estimators=400, random_state=42))
])

pipeline_svd = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,1), stop_words=ru_stopwords)),
    ('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(100)),
    ('clf', ExtraTreesClassifier(max_depth=60, min_samples_leaf=3, n_estimators=400, random_state=42))
])

pipeline_nmf = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,1), stop_words=ru_stopwords)),
    ('tfidf', TfidfTransformer()),
    ('svd', NMF(10)),
    ('clf', ExtraTreesClassifier(max_depth=60, min_samples_leaf=3, n_estimators=400, random_state=42))
])

In [428]:
metrics_bow, errors_bow = eval_table(data['comment_norm'], data['toxic'], pipeline_bow)
metrics_svd, errors_svd = eval_table(data['comment_norm'], data['toxic'], pipeline_svd)
metrics_nmf, errors_nmf = eval_table(data['comment_norm'], data['toxic'], pipeline_nmf)

In [429]:
metrics_bow

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.69,0.0,1.00,0.0,0.81,0.00
1.0,0.98,0.0,0.10,0.0,0.18,0.01
mean,0.84,0.0,0.55,0.0,0.50,0.00


In [430]:
metrics_svd

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.79,0.00,0.95,0.00,0.86,0.00
1.0,0.82,0.01,0.49,0.01,0.62,0.01
mean,0.80,0.00,0.72,0.00,0.74,0.00


In [431]:
metrics_nmf

,precision,precision_std,recall,recall_std,f1,f1_std
0.0,0.79,0.00,0.91,0.01,0.85,0.00
1.0,0.75,0.01,0.52,0.01,0.62,0.01
mean,0.77,0.00,0.72,0.01,0.74,0.00


In [432]:
errors_bow

array([[0.9988525 , 0.0011475 ],
       [0.90074629, 0.09925371]])

In [433]:
errors_svd

array([[0.94585854, 0.05414146],
       [0.50642285, 0.49357715]])

In [434]:
errors_nmf

array([[0.91445785, 0.08554215],
       [0.47928127, 0.52071873]])

<b>Лучший результат показал BOW + SVD</b>

<hr>

### Задание № 2 (6 баллов)

В Gensim тоже можно добавить нграммы и tfidf. Постройте 1 модель без них (как в семинаре) и еще 3 модели (1 с нграммами, 1 с tfidf и 1 с нграммами и с tfidf). Сранивте качество с помощью метрик (перплексия, когерентность) и на глаз. Определите лучшую модель. Для каждой модели выберите 1 самую красивую на ваш взгляд тему.

Используйте данные википедии из семинара. Можете взять поменьше данных, если все обучается долго.

Важное требование - получившиеся модели не должны быть совсем плохими. Если хороших тем не получается, попробуйте настроить гиперпараметры, отфильтровать словарь по-другому. 

Нграммы добавляются вот так (перед созданиеv словаря)

In [3]:
texts = open('wiki_data.txt', encoding='utf-8').read().splitlines()[:5000]
texts = ([normalize(text) for text in texts])

In [4]:
dictionary = gensim.corpora.Dictionary([text.split() for text in texts])

In [5]:
dictionary.filter_extremes(no_above=0.1, no_below=10)
dictionary.compactify()

In [6]:
corpus = [dictionary.doc2bow(text.split()) for text in texts]

In [7]:
lda = gensim.models.LdaModel(corpus, 
                             num_topics=100, 
                             alpha='asymmetric', 
                             id2word=dictionary, 
                             passes=10)

In [8]:
lda.print_topics()

[(98,
  '0.112*"банк" + 0.046*"кавказ" + 0.039*"дирижёр" + 0.028*"гимнастика" + 0.027*"чёрный" + 0.026*"ао" + 0.021*"радиостанция" + 0.021*"32" + 0.020*"край" + 0.019*"франция"'),
 (99,
  '0.025*"’" + 0.014*"брак" + 0.014*"слово" + 0.011*"язык" + 0.010*"ребёнок" + 0.008*"другой" + 0.008*"форма" + 0.007*"образ" + 0.007*"например" + 0.006*"случай"'),
 (97,
  '0.039*"самолёт" + 0.016*"авиационный" + 0.016*"полигон" + 0.015*"испытание" + 0.014*"воздушный" + 0.012*"аэродром" + 0.011*"вооружение" + 0.011*"полёт" + 0.009*"экипаж" + 0.009*"пилот"'),
 (96,
  '0.016*"система" + 0.008*"должный" + 0.006*"данные" + 0.006*"информация" + 0.006*"условие" + 0.006*"случай" + 0.005*"цель" + 0.005*"возможность" + 0.005*"для" + 0.005*"проект"'),
 (94,
  '0.048*"турнир" + 0.047*"финал" + 0.038*"раунд" + 0.034*"чемпионат" + 0.031*"выйти" + 0.023*"полуфинал" + 0.022*"четвертьфинал" + 0.020*"проиграть" + 0.019*"смочь" + 0.016*"сезон"'),
 (95,
  '0.074*"премия" + 0.056*"роль" + 0.051*"хороший" + 0.051*"фильм" +

In [9]:
def show_topics(lda_model, num_topics=num_topics)
    topics = []

        for topic_id, topic in lda.show_topics(num_topics=num_topics, formatted=False):
            topic = [word for word, _ in topic]
            topics.append(topic)
    return topics

def get_coherence(lda_model, texts, dictionary):
    topics = show_topics(lda_model, num_topics)
    coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=[text.split() for text in texts], 
                                                   dictionary=dictionary, coherence='c_v')
    
    coherence_val = coherence_model_lda.get_coherence()
    return coherence_val

In [11]:
print('Perplexity: ', np.exp2(-lda.log_perplexity(corpus)))

Perplixity:  269.93952343730285


In [13]:
print('Coherence: ', get_coherence(lda, texts, dictionary))

Coherence:  0.5944504122095


<b>Хороший пример:</b>

(98,
  '0.131*"германия" + 0.111*"немецкий" + 0.052*"лагерь" + 0.035*"больница" + 0.034*"заключить" + 0.029*"мировой" + 0.029*"берлин" + 0.025*"медицинский" + 0.023*"нацистский" + 0.022*"концлагерь"')
  
  (2,
  '0.041*"роль" + 0.027*"фильм" + 0.011*"американский" + 0.010*"играть" + 0.009*"актёр" + 0.009*"нью-йорк" + 0.008*"актриса" + 0.008*"сняться" + 0.007*"сыграть" + 0.007*"карьера"')

Теперь добавим н-граммы:

In [40]:
#texts = [text.split() for text in texts]
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.9) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

Н-граммы есть в корпусе, значит все ок:

In [41]:
[text for text in ngrammed_texts[:3]]

[['новостройка',
  'нижегородский',
  'область',
  'новостро́йка',
  'сельский',
  'посёлок',
  'дивеевский',
  'район',
  'нижегородский',
  'область',
  'входить',
  'состав',
  'сатисский',
  'сельсовет',
  'посёлок',
  'расположить',
  '12,5',
  'км',
  'юг',
  'село',
  'дивеево',
  '1',
  'км',
  'запад',
  'город',
  'саров',
  'право',
  'берег',
  'река',
  'вичкинза',
  'правый',
  'приток',
  'река',
  'сатис',
  'окружить',
  'смешанный',
  'лес',
  'соединить',
  'асфальтовый',
  'дорога',
  'посёлок',
  'цыгановка',
  '1,5',
  'км',
  'грунтовый',
  'просёлочный',
  'дорога',
  'посёлок',
  'сатис',
  '3,5',
  'км',
  'название',
  'новостройка',
  'являться',
  'сугубо',
  'официальный',
  'местный',
  'население',
  'использовать',
  'исключительно',
  'альтернативный',
  'название',
  'хитрый',
  'употребляться',
  'языковой',
  'оборот',
  '…',
  'хитрый',
  'ранее',
  'использовать',
  'название',
  'песчаный',
  'известковый',
  'основать',
  '1920-й',
  'год',
  'п

In [122]:
ngrammed_dictionary = gensim.corpora.Dictionary(ngrammed_texts)

In [123]:
ngrammed_dictionary.filter_extremes(no_above=0.1, no_below=5)
ngrammed_dictionary.compactify()

In [124]:
print(ngrammed_dictionary)

Dictionary(14055 unique tokens: ['1,2', '1,5', '12', '12,5', '14']...)


In [125]:
ngrammed_corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]

In [126]:
ngrammed_lda = gensim.models.LdaModel(ngrammed_corpus, 
                                      num_topics=100, 
                                      alpha='symmetric', 
                                      id2word=ngrammed_dictionary, 
                                      passes=15)

In [131]:
ngrammed_lda.print_topics()

[(12,
  '0.045*"юридический" + 0.043*"поэтому" + 0.026*"послужить" + 0.026*"оказаться" + 0.025*"керамика" + 0.024*"указывать" + 0.023*"самолёт" + 0.023*"1874" + 0.021*"нью-йорк" + 0.021*"переживать"'),
 (60,
  '0.058*"реформа" + 0.053*"1920" + 0.046*"способствовать" + 0.040*"йозеф" + 0.028*"давать" + 0.019*"регулирование" + 0.017*"техас" + 0.017*"вместе" + 0.016*"означать" + 0.014*"военачальник"'),
 (98,
  '0.077*"увлекаться" + 0.026*"стан" + 0.023*"приписать" + 0.020*"коридор" + 0.019*"венгерский" + 0.017*"иоанн" + 0.017*"алексеевич" + 0.015*"рассесть" + 0.015*"внешность" + 0.011*"спина"'),
 (14,
  '0.042*"продюсировать" + 0.033*"датировка" + 0.028*"закрытый" + 0.025*"содержаться" + 0.023*"1526" + 0.023*"прокуратура" + 0.023*"осуществить" + 0.022*"2005" + 0.017*"комиссариат" + 0.017*"кавалерийский"'),
 (35,
  '0.054*"поэт" + 0.052*"роберт" + 0.033*"уйти" + 0.022*"контакт" + 0.019*"перестать" + 0.018*"достаточно" + 0.017*"повстанец" + 0.016*"собранный" + 0.015*"декорация" + 0.013*"угро

In [128]:
print('Perplexity: ', np.exp2(-ngrammed_lda.log_perplexity(ngrammed_corpus)))

Perplexity:  267.6419379875327


In [129]:
print('Coherence:', get_coherence(ngrammed_lda, [' '.join(text) for text in ngrammed_texts], ngrammed_dictionary))

Coherence: 0.5906372721045241


Хороший пример:

(60,
  '0.058*"реформа" + 0.053*"1920" + 0.046*"способствовать" + 0.040*"йозеф" + 0.028*"давать" + 0.019*"регулирование" + 0.017*"техас" + 0.017*"вместе" + 0.016*"означать" + 0.014*"военачальник"')

Теперь добавим tfidf к корпусу c н-граммами:

In [132]:
tfidf = gensim.models.TfidfModel(ngrammed_corpus, id2word=ngrammed_dictionary)
ngrammed_tfidf_corpus = tfidf[ngrammed_corpus]

In [156]:
ngrammed_tfidf_lda = gensim.models.LdaModel(ngrammed_tfidf_corpus,
                                            40,
                                            alpha='symmetric',
                                            id2word=ngrammed_dictionary,
                                            passes=12)

In [157]:
ngrammed_tfidf_lda.print_topics()

[(4,
  '0.022*"гуйян" + 0.017*"боец" + 0.013*"зарабатывать" + 0.013*"дискриминация" + 0.010*"фантастика" + 0.010*"уважать" + 0.008*"исполнить" + 0.008*"филиал" + 0.007*"священнослужитель" + 0.007*"4-й"'),
 (23,
  '0.038*"легальный" + 0.016*"легион" + 0.009*"джим" + 0.008*"маркиз" + 0.006*"мартынов" + 0.006*"комендант" + 0.006*"сечение" + 0.006*"прекращение" + 0.006*"неофициальный" + 0.005*"воровство"'),
 (37,
  '0.017*"прохождение" + 0.013*"убивать" + 0.013*"проблемный" + 0.012*"средневековье" + 0.011*"ничто" + 0.010*"жестокий" + 0.009*"девочка" + 0.008*"призывать" + 0.007*"невский" + 0.007*"маркс"'),
 (39,
  '0.016*"нога" + 0.013*"эссекс" + 0.013*"беспрецедентный" + 0.011*"поездка" + 0.011*"холокост" + 0.008*"эксплуатационный" + 0.008*"тут" + 0.007*"суммарный" + 0.006*"безуспешно" + 0.005*"лабораторный"'),
 (8,
  '0.026*"лорд" + 0.014*"подозрение" + 0.007*"искусство" + 0.007*"сомнение" + 0.006*"аделаида" + 0.006*"1864" + 0.005*"узнать" + 0.005*"оскар" + 0.005*"60-й" + 0.005*"япония"')

In [142]:
print('Perplexity: ', np.exp2(-ngrammed_tfidf_lda.log_perplexity(ngrammed_tfidf_corpus)))

Perplexity:  18082125.771435328


In [150]:
print('Perplexity: ', np.exp2(-ngrammed_tfidf_lda.log_perplexity(ngrammed_corpus)))

Perplexity:  1008.5158559890915


In [159]:
print('Coherence: ', get_coherence(ngrammed_tfidf_lda, [' '.join(text) for text in ngrammed_texts], ngrammed_dictionary))

Coherence:  0.5906372721045241


Хорошие примеры:

  '0.013*"различие" + 0.010*"метр" + 0.008*"ключевой" + 0.008*"антенна" + 0.008*"италия" + 0.008*"закарпатский" + 0.007*"кредит" + 0.007*"зима" + 0.006*"квартал" + 0.006*"карпаты"'),

Теперь будем использовать только tfidf без н-грамм:

In [162]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
tfidf_corpus = tfidf[corpus]

In [163]:
tfidf_lda = gensim.models.LdaModel(tfidf_corpus, 100, alpha='symmetric', id2word=dictionary, passes=10)

In [84]:
tfidf_lda.print_topics()

[(4,
  '0.033*"дата" + 0.024*"петровский" + 0.020*"кардинал" + 0.016*"проведение" + 0.012*"трибуна" + 0.008*"1700" + 0.000*"василиевич" + 0.000*"ярославль" + 0.000*"денежный" + 0.000*"братский"'),
 (46,
  '0.027*"церемония" + 0.024*"таблица" + 0.024*"спортсменка" + 0.023*"54" + 0.023*"ален" + 0.019*"microsoft" + 0.019*"запись" + 0.018*"солнечный" + 0.017*"windows" + 0.016*"грузовой"'),
 (63,
  '0.077*"чемпионка" + 0.033*"wwe" + 0.015*"келли" + 0.011*"поединок" + 0.008*"чемпионский" + 0.000*"девушка" + 0.000*"шоу" + 0.000*"одержать" + 0.000*"титул" + 0.000*"победа"'),
 (14,
  '0.040*"гарнизонный" + 0.035*"юрий" + 0.023*"портрет" + 0.018*"тетерев" + 0.017*"кавалерийский" + 0.015*"национальность" + 0.012*"распределение" + 0.011*"башкортостан" + 0.010*"антонио" + 0.009*"уфа"'),
 (40,
  '0.020*"советский" + 0.020*"ленин" + 0.019*"орден" + 0.019*"фронт" + 0.014*"1-й" + 0.014*"аэропорт" + 0.013*"командир" + 0.013*"гвардейский" + 0.013*"ссср" + 0.011*"наградить"'),
 (48,
  '0.172*"уезд" + 0.07

In [85]:
print('Coherence: ', get_coherence(tfidf_lda, [' '.join(text) for text in texts], dictionary))

0.5632385193307283


In [166]:
print('Perplexity: ', np.exp2(-tfidf_lda.log_perplexity(tfidf_corpus)))

Perplexity:  76276958950.66371


In [168]:
print('Perplexity: ', np.exp2(-tfidf_lda.log_perplexity(corpus)))

Perplexity:  583125.311014991


Хорошие примеры:

  '0.020*"советский" + 0.020*"ленин" + 0.019*"орден" + 0.019*"фронт" + 0.014*"1-й" + 0.014*"аэропорт" + 0.013*"командир" + 0.013*"гвардейский" + 0.013*"ссср" + 0.011*"наградить"'),


Без нграмм модель работает гораздо лучше, и темы более качественные получаются. Хотя перплексия какая-то странная у tfidf моделей. Возможно проблема в десятичных значениях встречаемости токенов и словосочетаний. Но даже при подстановке обычного корпуса в формулу перплексии (не tfidf), получается плохое значение. Отпишите в комментарий пожалуйста, почему так. Хотя когеренция обладает хорошим значением, но все равно меньше чем у обычной модели на униграммах.